# Cleaning the Test Data

- I'm just going to quick clean up the test data that same way I did for the training data to be used later to make my predictions and submissiions. 

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings
import types
from ibm_botocore.client import Config
import ibm_boto3

warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
# The code was removed by Watson Studio for sharing.

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
df.shape

(1459, 80)

In [5]:
# Change YearBuilt into decades in order to plot it out easier

def construction_decade(row):
    if row['YearBuilt'] >= 1870 and row['YearBuilt'] < 1880:
        return '1870'
    elif row['YearBuilt'] >= 1880 and row['YearBuilt'] < 1890:
        return '1880'
    elif row['YearBuilt'] >= 1890 and row['YearBuilt'] < 1900:
        return '1890'
    elif row['YearBuilt'] >= 1900 and row['YearBuilt'] < 1910:
        return '1900'
    elif row['YearBuilt'] >= 1910 and row['YearBuilt'] < 1920:
        return '1910'
    elif row['YearBuilt'] >= 1920 and row['YearBuilt'] < 1930:
        return '1920'
    elif row['YearBuilt'] >= 1930 and row['YearBuilt'] < 1940:
        return '1930'
    elif row['YearBuilt'] >= 1940 and row['YearBuilt'] < 1950:
        return '1940'
    elif row['YearBuilt'] >= 1950 and row['YearBuilt'] < 1960:
        return '1950'
    elif row['YearBuilt'] >= 1960 and row['YearBuilt'] < 1970:
        return '1960'
    elif row['YearBuilt'] >= 1970 and row['YearBuilt'] < 1980:
        return '1970'
    elif row['YearBuilt'] >= 1980 and row['YearBuilt'] < 1990:
        return '1980'
    elif row['YearBuilt'] >= 1990 and row['YearBuilt'] < 2000:
        return '1990'
    elif row['YearBuilt'] >= 2000 and row['YearBuilt']:
        return '2000'
    else:
        return '0'
    
df['DecadeBuilt'] = df.apply(lambda row: construction_decade(row), axis=1)

In [6]:
df['DecadeBuilt'].head()

0    1960
1    1950
2    1990
3    1990
4    1990
Name: DecadeBuilt, dtype: object

## Fix Any Missing Data

In [9]:
# Find the percentage of missing values in each column
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageFinish,78,0.053461
GarageYrBlt,78,0.053461
GarageCond,78,0.053461
GarageQual,78,0.053461


In [13]:
# Fill in NaN values with "None" od "0"
df['PoolQC'].fillna('None', inplace=True)
df['MiscFeature'].fillna('None', inplace=True)
df['Alley'].fillna('None', inplace=True)
df['Fence'].fillna('None', inplace=True)
df['FireplaceQu'].fillna('None', inplace=True)
df['GarageYrBlt'].fillna('0', inplace=True)
df['BsmtExposure'].fillna('No', inplace=True)
df['MasVnrArea'].fillna('0', inplace=True)
df['MasVnrType'].fillna('None', inplace=True)
df['MasVnrArea'].astype('int', inplace=True).head()

0      0
1    108
2      0
3     20
4      0
Name: MasVnrArea, dtype: int64

In [11]:
# All NAN values in these 4 garage variables infers there is no garage. Change NAN to None. 
for col in ('GarageQual', 'GarageType', 'GarageCond', 'GarageFinish', 'GarageType'):
    df[col] = df[col].fillna('None')
    
# Basement variables with 'NAN' most likey don't have basements. Change NAN to None.
for col in ('BsmtFinType2','BsmtQual','BsmtCond','BsmtFinType1'):
    df[col] = df[col].fillna('None')

In [12]:
# NA values must be actual missing data. Group by neighborhood and impute the median LotFrontage.
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Fill the missing value with the mode.
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace=True)

In [16]:
# Check to make sure there are no missing values left.
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
MSZoning,4,0.002742
Functional,2,0.001371
Utilities,2,0.001371
BsmtHalfBath,2,0.001371
BsmtFullBath,2,0.001371
KitchenQual,1,0.000685
BsmtFinSF1,1,0.000685
Exterior1st,1,0.000685
Exterior2nd,1,0.000685
GarageCars,1,0.000685


### There are 15 more columns with NaN values in the Test data then there was in the Training data. I'll have to go through all of these columns now.

#### MSZoning

In [17]:
# Find the unique values
df['MSZoning'].unique()

array(['RH', 'RL', 'RM', 'FV', 'C (all)', nan], dtype=object)

In [18]:
# Fill in NaN values with "None"
df['MSZoning'].fillna('None', inplace=True)

In [19]:
# Check
df['MSZoning'].unique()

array(['RH', 'RL', 'RM', 'FV', 'C (all)', 'None'], dtype=object)

#### Functional

In [20]:
df['Functional'].unique()

array(['Typ', 'Min2', 'Min1', 'Mod', 'Maj1', 'Sev', 'Maj2', nan], dtype=object)

In [21]:
# Fill NaN with "None"
df['Functional'].fillna('None', inplace=True)

In [22]:
#Check
df['Functional'].unique()

array(['Typ', 'Min2', 'Min1', 'Mod', 'Maj1', 'Sev', 'Maj2', 'None'], dtype=object)

#### Utilities

In [23]:
df['Utilities'].unique()

array(['AllPub', nan], dtype=object)

In [24]:
# Fill NaN with 'NonPub'
df['Utilities'].fillna('NonPub', inplace=True)

In [25]:
# Check.
df['Utilities'].unique()

array(['AllPub', 'NonPub'], dtype=object)